In [ ]:
%cd /content
!pip install torchsde einops diffusers transformers accelerate peft timm kornia aiohttp
!apt install -qqy

!git clone https://github.com/comfyanonymous/ComfyUI /content/ComfyUI
!git clone https://github.com/ltdrdata/ComfyUI-Manager /content/ComfyUI/custom_nodes/ComfyUI-Manager

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-dev.sft  -d /content/ComfyUI/models/unet -o flux1-dev.sft
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/clip_l.safetensors -d /content/ComfyUI/models/clip -o clip_l.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/t5xxl_fp16.safetensors -d /content/ComfyUI/models/clip -o t5xxl_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/ae.sft -d /content/ComfyUI/models/vae -o ae.sft
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-redux-dev.safetensors -d /content/ComfyUI/models/style_models -o flux1-redux-dev.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/google/siglip-so400m-patch14-384/resolve/main/model.safetensors -d /content/ComfyUI/models/clip_vision -o clip_vision.safetensors

In [ ]:
%cd /content/ComfyUI

import os, shutil, json, requests, random, time
from urllib.parse import urlsplit

import torch
from PIL import Image
import numpy as np

from nodes import NODE_CLASS_MAPPINGS
from comfy_extras import nodes_flux, nodes_model_advanced, nodes_custom_sampler

UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
DualCLIPLoader = NODE_CLASS_MAPPINGS["DualCLIPLoader"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
CLIPVisionLoader = NODE_CLASS_MAPPINGS["CLIPVisionLoader"]()
LoadImage = NODE_CLASS_MAPPINGS["LoadImage"]()
StyleModelLoader =  NODE_CLASS_MAPPINGS["StyleModelLoader"]()

ModelSamplingFlux = nodes_model_advanced.NODE_CLASS_MAPPINGS["ModelSamplingFlux"]()
StyleModelApply = NODE_CLASS_MAPPINGS["StyleModelApply"]()
CLIPVisionEncode = NODE_CLASS_MAPPINGS["CLIPVisionEncode"]()
CLIPTextEncode = NODE_CLASS_MAPPINGS["CLIPTextEncode"]()
FluxGuidance = nodes_flux.NODE_CLASS_MAPPINGS["FluxGuidance"]()
RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()

with torch.inference_mode():
    unet = UNETLoader.load_unet("flux1-dev.sft", "default")[0]
    clip = DualCLIPLoader.load_clip("google_t5-v1_1-xxl_encoderonly-fp8_e4m3fn.safetensors", "clip_l.safetensors", "flux")[0]
    clip_vision = CLIPVisionLoader.load_clip("google--siglip-so400m-patch14-384/model.safetensors")[0]
    style_model = StyleModelLoader.load_style_model("flux1-redux-dev.safetensors")[0]
    vae = VAELoader.load_vae("ae.sft")[0]

def download_file(url, save_dir, file_name):
    os.makedirs(save_dir, exist_ok=True)
    file_suffix = os.path.splitext(urlsplit(url).path)[1]
    file_name_with_suffix = file_name + file_suffix
    file_path = os.path.join(save_dir, file_name_with_suffix)
    response = requests.get(url)
    response.raise_for_status()
    with open(file_path, 'wb') as file:
        file.write(response.content)
    return file_path

@torch.inference_mode()
def generate(input):
    values = input["input"]

    input_image1 = values['input_image1']
    input_image1 = download_file(url=input_image1, save_dir='/content/ComfyUI/input', file_name='input_image1')
    input_image2 = values['input_image2']
    input_image2 = download_file(url=input_image2, save_dir='/content/ComfyUI/input', file_name='input_image2')
    positive_prompt = values['positive_prompt']
    seed = values['seed']
    steps = values['steps']
    guidance = values['guidance']
    sampler_name = values['sampler_name']
    scheduler = values['scheduler']
    max_shift = values['max_shift']
    base_shift = values['base_shift']
    width = values['width']
    height = values['height']

    if seed == 0:
        random.seed(int(time.time()))
        seed = random.randint(0, 18446744073709551615)
    print(seed)

    image1 = LoadImage.load_image(input_image1)[0]
    image2 = LoadImage.load_image(input_image2)[0]
    conditioning_positive = CLIPTextEncode.encode(clip, positive_prompt)[0]
    conditioning_positive = FluxGuidance.append(conditioning_positive, guidance)[0]
    clip_vision_conditioning1 = CLIPVisionEncode.encode(clip_vision, image1)[0]
    style_vision_conditioning1 = StyleModelApply.apply_stylemodel(clip_vision_conditioning1, style_model, conditioning_positive)[0]
    clip_vision_conditioning2 = CLIPVisionEncode.encode(clip_vision, image2)[0]
    style_vision_conditioning2 = StyleModelApply.apply_stylemodel(clip_vision_conditioning2, style_model, style_vision_conditioning1)[0]
    unet_flux = ModelSamplingFlux.patch(unet, max_shift, base_shift, width, height)[0]
    noise = RandomNoise.get_noise(seed)[0]
    guider = BasicGuider.get_guider(unet_flux, style_vision_conditioning2)[0]
    sampler = KSamplerSelect.get_sampler(sampler_name)[0]
    sigmas = BasicScheduler.get_sigmas(unet_flux, scheduler, steps, 1.0)[0]
    latent_image = EmptyLatentImage.generate(width, height)[0]
    samples, _ = SamplerCustomAdvanced.sample(noise, guider, sampler, sigmas, latent_image)
    decoded = VAEDecode.decode(vae, samples)[0].detach()
    Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save(f"/content/flux.1-dev-redux-{seed}-tost.png")

    result = f"/content/flux.1-dev-redux-{seed}-tost.png"

    return result

In [ ]:
input = { 
        "input": {
        "input_image1": "https://files.catbox.moe/1ba4mh.png",
        "input_image2": "https://files.catbox.moe/mkcchl.jpg",
        "positive_prompt": "cute anime",
        "negative_prompt": "blurry",
        "seed": 0,
        "steps": 20,
        "guidance": 30,
        "sampler_name": "euler",
        "scheduler": "normal",
        "max_shift": 1.15,
        "base_shift": 0.50,
        "width": 1024,
        "height": 1024
    }
}
image = generate(input)
Image.open(image)